In [ ]:
# Moon Data Processing Notebook
### Beginning the process of processing the QAG Moon bounce data from Sep 10, 2021
#### Jon Richards - SETI Institute Quantum Astronomy Group
#### This code draws a waterfall plot from the data

In [9]:
import time

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import scipy.fftpack
from scipy.signal import argrelextrema

from datetime import datetime as dt

from pylab import rcParams
rcParams['figure.figsize'] = 15, 4

In [10]:
# Define the path to the data file we wish to process
data_file_path = "/Volumes/QAG Disk 1/data/moon_09102021/moon_2021-09-10T22_19_57.781495_1y"
#data_file_path = "/Volumes/QAG Backup/data/moon_09102021/moon_2021-09-10T22_19_57.781495_0x"
#data_file_path = "./data/data_1x"

In [11]:
START_TIME = 1631312409.1568253092447917 #This is UTC 2021-09-10 22:20:09
SAMPLES_PER_SEC = 1920000
BYTES_PER_SAMPLE = 4
SECS_OFFSET = 3

def seek_by_time(file, datetime_str):
    """Find the offset into the file for the given datetime and seek.
    Arguments:
        file - the open file
        datetime_str - A datetime in the form 2021-09-10 22:20:09, this if UTC
    Returns:
        int - the bytes skipped in the file.
    """
    d_start = dt.strptime("2021-09-10 22:20:09", '%Y-%m-%d %H:%M:%S') 
    d = dt.strptime(datetime_str, '%Y-%m-%d %H:%M:%S') 
    diff_secs = int(dt.timestamp(d) - dt.timestamp(d_start)) + SECS_OFFSET

    bytes_to_skip = diff_secs * SAMPLES_PER_SEC * BYTES_PER_SAMPLE
    
    #print(bytes_to_skip)
    
    if file != None:
        file.seek(bytes_to_skip)
    
    return bytes_to_skip

if seek_by_time(None, "2021-09-10 22:20:10")  != 7680000:
    print("WARNING: seek_by_time() did not return the correct offset. Please check!")


In [12]:
def readBlock(file, num_samples):
    """ 
    Read the next FFT sized block of 16-bit I/Q data from an opened file.
    
    Arguments:
        file - The reference to the data file, already opened.
        num_samples - The number of 16-bit I/Q samples to read in.
            Example, If you FFT SIZE is 1024, the num_samples should
            be 1024. 2048 16-bit samples (4096 bytes total) will be 
            read and unpacked from the file.
            
        Return - A Numpy array of "num_samples" Complex64 values.
            None if the read is past the end of the file.
    
    """
    # Read the bytes from the file
    data_raw = np.fromfile(file, dtype=np.int16, count=num_samples*2)
    
    # Create a Complex64 Numpy array to hold the values
    data = np.zeros((num_samples,), dtype=np.complex64)
    
    # Reassemble the int16 aray into the Complex64 array.
    # Use a try block to catch the condition where the end of the file
    # has been passed.
    try:
        data[:] = (data_raw[0::2]) + 1j*(data_raw[1::2])
    except:
        return None
    
    return data

In [13]:
def calFreqRangeForAxis(center_hz, bw_hz):
    
    """
    For the X axis tick mark labels, calculate the frequency min/max/center
    
    Arguments:
        center_hz - The center frequency of the signal, in Hertz.
        bw_hz - The bandwidth of the signal, in Hertz.
        
    Returns - A tuple of the 
        min frequency od the signal, in MHz
        max frequency od the signal, in MHz
        center frequency od the signal, in MHz
        
    """
    min_hz = center_hz - bw_hz/2.0
    max_hz = center_hz + bw_hz/2.0
    
    return (min_hz/1000000.0, max_hz/1000000., center_hz/1000000.)

In [108]:
def create_waterfall(filename, title1, title2, datetime_str, num_secs, start_chan, end_chan, output_image_name=None):
    
    """Create a waterfall plot
    Arguments:
        filename - The full path filename
        datetime_str - the time offset into the file
        num_secs - number of seconds to process
        start_chan - the start frequency bin
        end_chan - the end frequency bin
        output_image_name- Optional, save an image of the plot
    """
    
    # Perform the FFT and store the result for later plotting.
    # The FFT result will be accumulated in a one dimensional Numpy array
    # of length equal to the FFT_SIZE

    # Define some constants. Here you may wish to change the FFT_SIZE
    FFT_SIZE = 1960000
    CENTER_FREQ_HZ = 1296000000
    BANDWIDTH_HZ   = 1920000
    MHZ = 1048576.0
    NUM_CHANS = end_chan - start_chan

    # Create an array to hold the data for each horizontal line of the waterfall
    bins = []

    # How many FFT_SIZE chunks to process. Ideally we would like to
    # process the entire channel, but during code development we may
    # wish to read in a smaller number. Reading in the large data files
    # takes a lot of time.
    NUM = num_secs

    # Open the file
    file = open(filename, "rb")

    # Seek
    seek_by_time(file, datetime_str)
    
    # Create an array to hold the accumulation for each bin
    fft_acc = np.zeros((end_chan - start_chan, ))

    # Keep a counter of the number of FFT_SIZE length chunks we processed.
    count = 0

    # Loop through the file. 
    for i in range(NUM):

        #print(i)

        # Read the next block of data from the file
        d = readBlock(file, FFT_SIZE)

        # If None is retruned, this signifies we read to the end of the file.
        # Break out of the loop.
        if d is None:
            print("d is NONE")
            break

        yf = scipy.fftpack.fft(d, n=FFT_SIZE, axis=0)
        
        #Add the results of the FFTp to the fft_acc Numpy array
        np.add(np.abs(yf[start_chan:end_chan]), fft_acc, out=fft_acc)
        
        #print(np.abs(yf[start_chan:start_chan+1]), fft_acc[0:1])
        
        xx = np.abs(yf).tolist()
        yy = scipy.fft.fftshift(xx)
        bins.append(yy[start_chan:end_chan])
        
        

        # Draw a plot of the results

    fig, ax = plt.subplots(1,2)
    #fig, ax = plt.subplots(1,2,figsize=(72/8, 60/4))

    #ax.set_ylim(0, 40)

    ax[0].imshow(bins)
    
    fig.suptitle(title1, fontsize=18)
    ax[0].set_title(title2)
    
    # Make the Y axis be log
    ax[1].set_yscale('log')
    
    # A strange artifact of the FFTp process is that the upper and lower
    # halved are swapped. The fftshift swaps them back to the correct order.
    fft_acc_temp = scipy.fft.fftshift(fft_acc) 
    fft_acc = fft_acc_temp
    
    chan_min_freq_mhz = (CENTER_FREQ_HZ - 1236524.0)/MHZ
    
    freqs_for_xaxis = [(chan_min_freq_mhz + (f*(BANDWIDTH_HZ/FFT_SIZE))/MHZ) for f in range(NUM_CHANS)]

    ax[1].plot(freqs_for_xaxis, fft_acc)


    if output_image_name != None:
        plt.savefig(output_image_name)
        
    # Show!
    #plt.show()
    plt.close(fig)


In [110]:
create_waterfall(data_file_path, "3, -3", "  ", "2021-09-10 23:05:00", 60, 1236524, 1236622, "test5")
#create_waterfall(data_file_path, "2021-09-10 22:24:00", 4*60, 1200000, 1250000, None)

In [ ]:
print(len(bins[119]))

In [111]:
# Create a list of the schedue
schedule = []

schedule.append([-3,3,'2021-09-10 23:05:00'])
schedule.append([-6,6,'2021-09-10 23:07:00'])
schedule.append([-9,9,'2021-09-10 23:09:00'])
schedule.append([-10,10,'2021-09-10 23:11:00'])
schedule.append([-11,11,'2021-09-10 23:13:00'])
schedule.append([-13,13,'2021-09-10 23:15:00'])
schedule.append([-3,6,'2021-09-10 23:17:00'])
schedule.append([-3,9,'2021-09-10 23:19:00'])
schedule.append([-3,10,'2021-09-10 23:21:00'])
schedule.append([-3,11,'2021-09-10 23:23:00'])
schedule.append([-3,13,'2021-09-10 23:25:00'])
schedule.append([-6,3,'2021-09-10 23:27:00'])
schedule.append([-6,9,'2021-09-10 23:29:00'])
schedule.append([-6,10,'2021-09-10 23:31:00'])
schedule.append([-6,11,'2021-09-10 23:33:00'])
schedule.append([-6,13,'2021-09-10 23:35:00'])
schedule.append([-9,3,'2021-09-10 23:37:00'])
schedule.append([-9,6,'2021-09-10 23:39:00'])
schedule.append([-9,10,'2021-09-10 23:41:00'])
schedule.append([-9,11,'2021-09-10 23:43:00'])
schedule.append([-9,13,'2021-09-10 23:45:00'])
schedule.append([-10,3,'2021-09-10 23:47:00'])
schedule.append([-10,6,'2021-09-10 23:49:00'])
schedule.append([-10,9,'2021-09-10 23:51:00'])
schedule.append([-10,11,'2021-09-10 23:53:00'])
schedule.append([-10,13,'2021-09-10 23:55:00'])
schedule.append([-11,3,'2021-09-10 23:57:00'])
schedule.append([-11,6,'2021-09-10 23:59:00'])
schedule.append([-11,9,'2021-09-11 00:01:00'])
schedule.append([-11,10,'2021-09-11 00:03:00'])
schedule.append([-11,13,'2021-09-11 00:05:00'])
schedule.append([-13,3,'2021-09-11 00:07:00'])
schedule.append([-13,6,'2021-09-11 00:09:00'])
schedule.append([-13,9,'2021-09-11 00:11:00'])
schedule.append([-13,10,'2021-09-11 00:13:00'])
schedule.append([-13,11,'2021-09-11 00:15:00'])
schedule.append([-10,10,'2021-09-11 00:17:00'])
schedule.append([-9,9,'2021-09-11 00:19:00'])
schedule.append([-8,8,'2021-09-11 00:21:00'])

#print(schedule)
print(len(schedule))




39


In [113]:
count = 0
for sched in schedule:
    
    count = count + 1
    print(count)
    
    offset1 = sched[0]
    offset2 = sched[1]
    d = sched[2]
    
    title = f"{offset1}, {offset2}"
    filename = f"images/offsets_{offset1},{offset2}_t{d.replace(':', '').replace(' ','').replace('-', '')}"
    print(title, filename)
    create_waterfall(data_file_path, title, "  ", d, 60, 1236524, 1236622, filename)

1
-3, 3 images/offsets_-3,3_t20210910230500
2
-6, 6 images/offsets_-6,6_t20210910230700
3
-9, 9 images/offsets_-9,9_t20210910230900
4
-10, 10 images/offsets_-10,10_t20210910231100
5
-11, 11 images/offsets_-11,11_t20210910231300
6
-13, 13 images/offsets_-13,13_t20210910231500
7
-3, 6 images/offsets_-3,6_t20210910231700
8
-3, 9 images/offsets_-3,9_t20210910231900
9
-3, 10 images/offsets_-3,10_t20210910232100
10
-3, 11 images/offsets_-3,11_t20210910232300
11
-3, 13 images/offsets_-3,13_t20210910232500
12
-6, 3 images/offsets_-6,3_t20210910232700
13
-6, 9 images/offsets_-6,9_t20210910232900
14
-6, 10 images/offsets_-6,10_t20210910233100
15
-6, 11 images/offsets_-6,11_t20210910233300
16
-6, 13 images/offsets_-6,13_t20210910233500
17
-9, 3 images/offsets_-9,3_t20210910233700
18
-9, 6 images/offsets_-9,6_t20210910233900
19
-9, 10 images/offsets_-9,10_t20210910234100
20
-9, 11 images/offsets_-9,11_t20210910234300
21
-9, 13 images/offsets_-9,13_t20210910234500
22
-10, 3 images/offsets_-10,3_t20